In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import math

In [2]:
a_1 = -1.1693673371945799
b_1 = 51.41919388017123
c_1 = 5.954712391658841
d_1 = 1.0045788520232413
params = [a_1, b_1, c_1, d_1]
data = [a_1, b_1, c_1, d_1]
h = .01
step_size = h
tolerance = .1



In [8]:
def next_erk(step_size, her2, akt, erk, k_1, k_2, deg_1):
    K1 = step_size*(k_1 * her2 - k_2 * akt * erk - deg_1 * erk)
    K2 = step_size*(k_1 * her2 - k_2 * akt * (erk + K1) - deg_1 * (erk + K1))
    return erk + ((K1+K2)/2)

def next_akt(step_size, her2, akt, k_3, deg_2):
    K1 = step_size*(k_3 * her2 - deg_2 * akt)
    K2 = step_size*(k_3 * her2 - deg_2 * (akt + K1))
    return akt + ((K1+K2)/2)


def next_il1(step_size, erk, p_nfkb, il1, k_4, k_5, deg_3):
    K1 = step_size*(k_4 * erk + k_5 * p_nfkb - deg_3 * il1)
    K2 = step_size*(k_4 * erk + k_5 * p_nfkb - deg_3 * (il1 + K1))
    return il1 + ((K1+K2)/2)


def next_ikk(step_size, akt, il1, ikk, k_6, k_7, deg_4):
    K1 = step_size*(k_6 * akt + k_7 * il1 - deg_4 * ikk)
    K2 = step_size*(k_6 * akt + k_7 * il1 - deg_4 * (ikk + K1))
    return ikk + ((K1+K2)/2)


def next_ikb_nfkb(step_size, ikk, ikb_nfkb, ikb, p_nfkb, k_8, k_9):
    K1 = step_size*(-1 * k_8 * ikk * ikb_nfkb + k_9 * ikb * p_nfkb)
    K2 = step_size*(-1 * k_8 * ikk * (ikb_nfkb + K1) + k_9 * ikb * p_nfkb)
    return ikb_nfkb + ((K1+K2)/2)


def next_p_ikb(step_size, ikk, ikb_nfkb, p_ikb, k_8, deg_5):
    K1 = step_size*(k_8 * ikk * ikb_nfkb - deg_5 * p_ikb)
    K2 = step_size*(k_8 * ikk * ikb_nfkb - deg_5 * (p_ikb + K1))
    return p_ikb + ((K1+K2)/2)


def next_p_nfkb(step_size, ikk, ikb_nfkb, ikb, p_nfkb, k_8, k_9):
    K1 = step_size*(k_8 * ikk * ikb_nfkb - k_9 * ikb * p_nfkb)
    K2 = step_size*(k_8 * ikk * ikb_nfkb - k_9 * ikb * (p_nfkb + K1))
    return p_nfkb + ((K1+K2)/2)


def next_ikb(step_size, p_nfkb, ikb, k_9, k_10):
    K1 = step_size*(k_10 * p_nfkb - k_9 * ikb * p_nfkb)
    K2 = step_size*(k_10 * p_nfkb - k_9 * (ikb + K1) * p_nfkb)
    return ikb + ((K1+K2)/2)



In [9]:
def RK2(step_size, t_start, t_end, start_values, k_values, deg_values):
    steps = int((t_end - t_start) / step_size)

    stored_erk = [start_values[1]]
    stored_akt = [start_values[2]]
    stored_il1 = [start_values[3]]
    stored_ikk = [start_values[4]]
    stored_ikb_nfkb = [start_values[5]]
    stored_p_ikb = [start_values[6]]
    stored_p_nfkb = [start_values[7]]
    stored_ikb = [start_values[8]]

    stored_vals = [[t_start, stored_p_nfkb[0]]]
    curr = 0 
    
    for i in range(steps):
        stored_erk.append(next_erk(step_size, start_values[0], stored_akt[i], stored_erk[i], k_values[0], k_values[1], deg_values[0]))
        stored_akt.append(next_akt(step_size, start_values[0], stored_akt[i], k_values[2], deg_values[1]))
        stored_il1.append(next_il1(step_size, stored_erk[i], stored_p_nfkb[i], stored_il1[i], k_values[3], k_values[4], deg_values[2]))
        stored_ikk.append(next_ikk(step_size, stored_akt[i], stored_il1[i], stored_ikk[i], k_values[5], k_values[6], deg_values[3]))
        stored_ikb_nfkb.append(next_ikb_nfkb(step_size, stored_ikk[i], stored_ikb_nfkb[i], stored_ikb[i], stored_p_nfkb[i], k_values[7], k_values[8]))
        stored_p_ikb.append(next_p_ikb(step_size, stored_ikk[i], stored_ikb_nfkb[i], stored_p_ikb[i], k_values[7], deg_values[0]))
        stored_p_nfkb.append(next_p_nfkb(step_size, stored_ikk[i], stored_ikb_nfkb[i], stored_ikb[i], stored_p_nfkb[i], k_values[7], k_values[8]))
        stored_ikb.append(next_ikb(step_size, stored_p_nfkb[i], stored_ikb[i], k_values[8], k_values[9]))

        curr = round(curr + step_size, 12)
        stored_vals.append([curr, stored_p_nfkb[i + 1]])
    return stored_vals


def E2(step_size, t_start, t_end, start_vals, k_vals, deg_vals):
    RK2_approx = RK2(step_size, t_start , t_end, start_vals, k_vals, deg_vals)
    max_error = 0
    iter = 0
    for i in range(len(RK2_approx)):
        while RK2_approx[i][0] != actual_list[iter][0]:
            iter += 1
            if iter >= len(actual_list):
                print("Exit due to error")
                sys.exit()
        max_error = max(max_error, abs(RK2_approx[i][1]-actual_list[iter][1]))
    return max_error

def order(step_size, t_start, t_end, start_vals, k_vals, deg_vals):
    #print(f"h is {h}")
    h = step_size
    h_new = float(h/2) #h_new is the new step-size, which is 1/2 of the original
    #print(f"h/2 is {h_new}")
    e2 = E2(h_new, 0,1,start_vals, k_vals, deg_vals) #Calculates the new error based on the new step-size
    #print(f"The max error, e2, with {h_new} is {e2}")
    e1 = E2(h, 0 ,1 , start_vals, k_vals, deg_vals)  #Calculates the old error based on the orignal step-size
   # print(f"The max error, e1, with {h} is {e1}")
    ratio = float(e2/e1)
    #print(f"ratio of the error is {ratio}")
    n = math.log2(abs(ratio))  #Calculates the order, n
    return -n

def convergence_case_1(step_size, t_start, t_end, start_vals, k_vals, deg_vals):
    print("\nFor max error")
    print(f"Order with h = {step_size} and h/2 = {step_size / 2} is: {order(step_size, t_start, t_end, start_vals, k_vals, deg_vals)}")
    step_size = step_size / 2
    print(f"Order with h = {step_size} and h/2 = {step_size / 2} is: {order(step_size, t_start, t_end, start_vals, k_vals, deg_vals)}")
    step_size = step_size / 2
    print(f"Order with h = {step_size} and h/2 = {step_size / 2} is: {order(step_size, t_start, t_end, start_vals, k_vals, deg_vals)}")
    step_size = step_size / 2
    print(f"Order with h = {step_size} and h/2 = {step_size / 2} is: {order(step_size, t_start, t_end, start_vals, k_vals, deg_vals)}")
    step_size = step_size / 2
    print(f"Order with h = {step_size} and h/2 = {step_size / 2} is: {order(step_size, t_start, t_end, start_vals, k_vals, deg_vals)}")


def main(h):
    # randomize these start values
    iter = 0
    while True:
        iter +=1
        her2_start = random.uniform(0,.5)
        erk_start = random.uniform(0,1)
        akt_start = random.uniform(0,1)
        il1_start = random.uniform(0,1)
        ikk_start = random.uniform(0,1)
        ikb_nfkb_start = random.uniform(0,1)
        p_ikb_start = random.uniform(0,1)
        p_nfkb_start = 1
        ikb_start = random.uniform(0,1)

        start_values = [her2_start, erk_start, akt_start, il1_start, ikk_start, ikb_nfkb_start, p_ikb_start, p_nfkb_start, ikb_start]
        #                 0,         1            2           3              4        5         6                 7            8
        k_1 = random.uniform(0,5)
        k_2 = random.uniform(0,5)
        k_3 = random.uniform(0,5)
        k_4 = random.uniform(0,5)
        k_5 = random.uniform(0,5)
        k_6 = random.uniform(0,10)
        k_7 = random.uniform(0,10)
        k_8 = random.uniform(0,5)
        k_9 = random.uniform(0,5)
        k_10 = random.uniform(0,15)
        k_values = [k_1, k_2, k_3, k_4, k_5, k_6, k_7, k_8, k_9, k_10]

        deg_1 = random.uniform(0,5)
        deg_2 = random.uniform(0,5)
        deg_3 = random.uniform(0,5)
        deg_4 = random.uniform(0,5)
        deg_5 = random.uniform(0,10)
        deg_values = [deg_1, deg_2, deg_3, deg_4, deg_5]


        da_list = RK2(h, 0, 1, start_values, k_values, deg_values)
        p_nfkb_actual = actual(data = params, step_size = h)
        t_vals = np.linspace(0,1,int((1/h) + 1))
        # print(start_values)
        # print(k_values)
        # print(deg_values)
        # print(da_list)
        if iter % 10000 == 0: 
            print(iter)
        #if abs(da_list[1000] - p_nfkb_actual[1000]) < .01 and abs(da_list[500] - p_nfkb_actual[500]) < .01:
        #    graphData(t_vals, p_nfkb_actual, da_list)
        #    print("Start Values", start_values)
        #    print("K vals", k_values)
        #    print("Deg vals", deg_values)
        #    break

        difference = []
        for i in range(len(da_list)):
            difference.append(abs(da_list[i] - p_nfkb_actual[i]))
        max_dif = max(difference)
        if max_dif < tolerance:
            graphData(t_vals, p_nfkb_actual, da_list)
            print("Start Values", start_values)
            print("K vals", k_values)
            print("Deg vals", deg_values)
            break
        

In [13]:
her2_start = 0.211
erk_start = .81
akt_start = 0.138
il1_start = 0.807
ikk_start = 0.516
ikb_nfkb_start = .099
p_ikb_start = 0.145
p_nfkb_start = .99
ikb_start = .1838

start_values = [her2_start, erk_start, akt_start, il1_start, ikk_start, ikb_nfkb_start, p_ikb_start, p_nfkb_start, ikb_start]
#                 0,         1            2           3          4        5               6                 7          8
k1 = 1.67
k2 = .023
k3 = 2.36
k4 = 2.44
k5 = 2.5
k6 = 5.87
k7 = 7.16
k8 = 4.813
k9 = 0.384
k10 = 7.68
k_values = [k1, k2, k3, k4, k5, k6, k7, k8, k9, k10]

deg_1 = 1.06
deg_2 = 1.23
deg_3 = 3.27
deg_4 = 1.035
deg_5 = 5.439
deg_values = [deg_1, deg_2, deg_3, deg_4, deg_5]

actual_list = RK2(0.0000001, 0, 1, start_values, k_values, deg_values)

In [14]:
len(actual_list)

10000001

In [15]:
convergence_case_1(step_size, 0, 1, start_values, k_values, deg_values)


For max error
Order with h = 0.01 and h/2 = 0.005 is: 0.9843200215287687
Order with h = 0.005 and h/2 = 0.0025 is: 0.9921961901745444
Order with h = 0.0025 and h/2 = 0.00125 is: 0.9961033995484354
Order with h = 0.00125 and h/2 = 0.000625 is: 0.9981449232013512
Order with h = 0.000625 and h/2 = 0.0003125 is: 0.9992474492155382
